<a href="https://colab.research.google.com/github/imammarzuki/Eksperimen/blob/main/HybridCF_CBF_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162986 sha256=27aa2bcaf8a7f4c4b05548d97734fd7ba0e6c478e157f9eb2b49fca1be2c5067
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import cross_validate, train_test_split
from surprise.accuracy import rmse, mae
from surprise import accuracy
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

In [3]:
data = {
    'user_id': ['1', '1', '2', '2', '3', '3', '4', '4'],
    'place_id': ['101', '102', '101', '103', '102', '104', '101', '104'],
    'place_name': ['Place A', 'Place B', 'Place C', 'Place D', 'Place E', 'Place F', 'Place G', 'Place H'],
    'description': ['beautiful garden view', 'lake view near mountains', 'urban center shopping district',
                    'quiet woodland area', 'historic town center', 'lively night district', 'peaceful seaside area',
                    'scenic rural landscape'],
    'price': [120, 150, 200, 90, 80, 220, 130, 110],
    'rating1': [5, 4, 5, 2, 3, 4, 1, 2],
    'rating2': [3, 3, 4, 1, 2, 5, 1, 2]
}

df = pd.DataFrame(data)
# Menggabungkan rating menjadi satu nilai dengan rata-rata
df['average_rating'] = df[['rating1', 'rating2']].mean(axis=1)

# Mengkonversi ke format yang diterima oleh Scikit-Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'place_id', 'average_rating']], reader)

In [4]:
# Membagi data menjadi train dan test set
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

# Menggunakan KNNBasic dengan kesamaan cosine
algo = KNNBasic(sim_options={'name': 'cosine', 'user_based': True})
algo.fit(trainset)

# Membuat prediksi pada test set
predictions = algo.test(testset)

# Menampilkan hasil prediksi
for prediction in predictions:
    print(prediction)

Computing the cosine similarity matrix...
Done computing similarity matrix.
user: 2          item: 103        r_ui = 1.50   est = 3.50   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 4          item: 101        r_ui = 1.00   est = 3.50   {'was_impossible': True, 'reason': 'Not enough neighbors.'}


In [5]:
# Menghitung RMSE dan MAE
print("RMSE: ", accuracy.rmse(predictions))
print("MAE: ", accuracy.mae(predictions))

# Menghitung Confusion Matrix
actual_ratings = np.array([pred.r_ui for pred in predictions])
predicted_ratings = np.array([pred.est for pred in predictions])
cm = confusion_matrix(actual_ratings.round(), predicted_ratings.round(), labels=[1, 2, 3, 4, 5])
print("Confusion Matrix:\n", cm)

RMSE: 2.2638
RMSE:  2.2638462845343543
MAE:  2.2500
MAE:  2.25
Confusion Matrix:
 [[0 0 0 1 0]
 [0 0 0 1 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
